# **0. 구글 드라이브 임포트**

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [ ]:
# 드라이브 위치 확인
%cd /content/gdrive/MyDrive/Colab Notebooks/PITer

/content/gdrive/MyDrive/Colab Notebooks/PITer


In [ ]:
# How to check HF access token: hugging face -> select account at top right -> settings -> access token
import huggingface_hub
huggingface_hub.login()

# **1. Install and Import**

In [ ]:
!pip install datasets
!pip install transformers accelerate bitsandbytes peft trl
!pip install --upgrade transformers trl
!pip install chardet
!pip install jsonlines
!pip install torch
!pip install gguf

In [ ]:
import torch
import pandas as pd
import numpy as np
import warnings
import time
import chardet

import json
import jsonlines

from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    DataCollatorForLanguageModeling,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
    logging,
)

from peft import LoraConfig, get_peft_model
from trl import SFTTrainer
from huggingface_hub import notebook_login
from datasets import load_dataset

import sys
import logging
import datasets
import transformers

logger = logging.getLogger(__name__)

# **1-1. 사전 모델과 새로 만들 모델 설정**

In [ ]:
# 허깅페이스 레포 기준으로 설정
base_model = "LGAI-EXAONE/EXAONE-3.0-7.8B-Instruct"

# 내가 만든 것
new_model = "LINKER98/CareerBoost-EXAONE3.0"

# **1-2. 4비트 양자화**

In [ ]:
# 현재 사용 중인 GPU의 주요 아키텍처 버전을 반환 8버전 이상 시 bfloat16 활용
if torch.cuda.get_device_capability()[0] >= 8:
    !pip install -qqq flash-attn
    attn_implementation = "flash_attention_2"
    torch_dtype = torch.bfloat16
else:
    attn_implementation = "eager"
    torch_dtype = torch.float16

# BitsAndBytesConfig 객체활용 양자화 설정
# 모델을 4비트 양자화하여 로드
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch_dtype,
    bnb_4bit_use_double_quant=False,
)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 73.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [ ]:
# 모델 로딩 (초기에 다운로드 받을 때 [구글 드라이브에 로컬 모델 없을 때!])
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=quant_config,
    # torch_dtype=torch.bfloat16,
    # trust_remote_code=True,
    device_map={"": 0} # 단일 GPU 기준 - 코랩은 단일 GPU
    # device_map="auto"
)
model.config.use_cache = False
model.config.pretraining_tp = 1

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/878 [00:00<?, ?B/s]

The repository for LGAI-EXAONE/EXAONE-3.0-7.8B-Instruct contains custom code which must be executed to correctly load the model. You can inspect the repository content at https://hf.co/LGAI-EXAONE/EXAONE-3.0-7.8B-Instruct.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


configuration_exaone.py:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/LGAI-EXAONE/EXAONE-3.0-7.8B-Instruct:
- configuration_exaone.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


...

model-00005-of-00007.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00006-of-00007.safetensors:   0%|          | 0.00/4.83G [00:00<?, ?B/s]

model-00007-of-00007.safetensors:   0%|          | 0.00/1.68G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

# **1-3. 사전 모델 로드 및 모델 토큰나이즈**

In [ ]:
# 토크나이저 설정
tokenizer = AutoTokenizer.from_pretrained(base_model)  # 실제 사용 모델의 이름으로 변경하세요.
tokenizer.padding_side = "right"
EOS_TOKEN = tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/70.7k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.93M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.22M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/4.96M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/563 [00:00<?, ?B/s]

# **안쓰는 코드**

In [ ]:
from transformers import AutoModelForCausalLM

# 모델 로드 (trust_remote_code=True 옵션 추가)
newmodel = AutoModelForCausalLM.from_pretrained("LINKER98/CareerBoost-EXAONE3.0", trust_remote_code=True)

newtokenizer = AutoTokenizer.from_pretrained("LINKER98/CareerBoost-EXAONE3.0")


In [ ]:
# 로컬 드라이브 (구글)에 저장할때 쓰는 코드 한 번만 실행!

from transformers import AutoModelForCausalLM, AutoTokenizer

# 모델 ID 설정
model = "LGAI-EXAONE/EXAONE-3.0-7.8B-Instruct"

# 모델과 토크나이저 로드
model = AutoModelForCausalLM.from_pretrained(model)
tokenizer = AutoTokenizer.from_pretrained("LGAI-EXAONE/EXAONE-3.0-7.8B-Instruct")

# 로컬에 저장할 경로 설정
save_directory = "./origin"

# 모델과 토크나이저 저장
model.save_pretrained(save_directory)
tokenizer.save_pretrained(save_directory)


In [ ]:
# Google Drive에 저장된 모델 로드
from transformers import AutoModelForCausalLM, AutoTokenizer
from google.colab import drive

# 저장된 모델 경로 (Google Drive의 origin 폴더)
model_path = "/content/gdrive/MyDrive/Colab Notebooks/PITer/origin"

# 모델과 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(model_path, trust_remote_code=True)

# 모델 로드 확인
print("Model and tokenizer successfully loaded from:", model_path)

# **2. Data Preprecessing and Tokenuzer**

In [ ]:
pwd

'/content/gdrive/MyDrive/Colab Notebooks/PITer'

In [ ]:
# 데이터셋 패스 정하기
dataPath = "/content/gdrive/MyDrive/Colab Notebooks/PITer/datasets/modifyData.csv"

In [ ]:
dataPath = load_dataset("csv", data_files=dataPath)

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
train_data = dataPath

In [ ]:
train_data

DatasetDict({
    train: Dataset({
        features: ['0'],
        num_rows: 5000
    })
})

In [ ]:
from datasets import DatasetDict

# 기존 데이터셋에서 'train' 부분을 추출
train_dataset = train_data['train']

# 무작위로 5,000개의 샘플 선택
reduced_train_dataset = train_dataset.shuffle(seed=24).select(range(5000))

# 새로운 DatasetDict 생성
reduced_dataset_dict = DatasetDict({'train': reduced_train_dataset})

# 결과 확인
print(reduced_dataset_dict)


DatasetDict({
    train: Dataset({
        features: ['0'],
        num_rows: 5000
    })
})


In [ ]:
# CSV 파일로 저장
reduced_train_dataset.to_csv('modifyData.csv', index=False)

Creating CSV from Arrow format:   0%|          | 0/5 [00:00<?, ?ba/s]

1118407

In [ ]:
# 저장된 CSV 파일 확인
import pandas as pd

df = pd.read_csv('modifyData.csv')
print(df.head())

                                                   0
0  ### Instruction: 양념치킨하고 후라이드 치킨을 반반씩 시켜서 후라이드 ...
1  ### Instruction: 장래희망이 뭐냐니깐! 이 문장을 정중한 말투로 바꿔줘...
2  ### Instruction: 당연ㅋㅋ 누가 반려동물 잡아먹음? 이 문장을 정중한 ...
3  ### Instruction: ㅇㅇ 항상 빨간 건 아니고 인터뷰 맞춰서 염색한 거임...
4  ### Instruction: 너 밥 먹고 나서 양치 안하지? 이 문장을 정중한 말...


In [ ]:
# 마지막 데이터 확인
last_data = train_data['train'][-1]
print(last_data)
info_data = train_data['train'][3]
print(info_data)
info_data2 = train_data['train'][3240]
print(info_data2)
info_data3 = train_data['train'][450]
print(info_data3)
info_data4 = train_data['train'][4998]
print(info_data4)

{'0': '### Instruction: 난 지금 뮤지컬 보고 오는 길임 이 문장을 정중한 말투로 바꿔줘.\n\n### Response: 저는 지금 뮤지커을 보고 돌아오는 길입니다.[|endofturn|]'}
{'0': '### Instruction: ㅇㅇ 항상 빨간 건 아니고 인터뷰 맞춰서 염색한 거임 이 문장을 정중한 말투로 바꿔줘.\n\n### Response: 네, 항상 빨간 건 아니고 인터뷰에 맞춰서 염색한 것입니다.[|endofturn|]'}
{'0': '### Instruction: 허나 짐의 점심을 조금 주겠소. 새우 부리토요. 이 문장을 정중한 말투로 바꿔줘.\n\n### Response: 하지만 제 점심을 조금 드릴게요. 새우 부리토입니다.[|endofturn|]'}
{'0': '### Instruction: 진짜 괘씸하고 위험한 장난이다.. ㅠ 이 문장을 정중한 말투로 바꿔줘.\n\n### Response: 매우 괘씸하고 위험한 장난을 쳤네요.[|endofturn|]'}
{'0': '### Instruction: 무엇보다, 충분히 먹었으면 과거로 되돌아가지 않으면 될 일이지. 무한 개의 치킨 샌드위치를 원하지만 영원히 먹고싶은 마음은 없어~╮(╯▽╰)╭ 이 문장을 정중한 말투로 바꿔줘.\n\n### Response: 무엇보다, 충분히 먹었으면 과거로 되돌아가지 않으면 될 일입니다. 무한개의 치킨 샌드위치를 원하지만 영원히 먹고싶은 마음은 없어요.[|endofturn|]'}


In [ ]:
from datasets import Dataset
from transformers import AutoTokenizer
import pandas as pd

# 데이터셋 로드 및 전처리
data = pd.read_csv('datasets/modifyData.csv')
# 데이터프레임에서 리스트 형태로 변환 후 Huggingface Dataset으로 변환
dataset = Dataset.from_pandas(data)

# 데이터셋에서 기존 열 이름 가져오기
existing_column_name = dataset.column_names[0]

# 기존 열 이름을 사용하여 formatting_prompts_func 정의
def formatting_prompts_func(examples):
    combined_texts = examples[existing_column_name]
    texts = []
    for combined_text in combined_texts:
        # "### Instruction:" 이후 내용을 분리합니다.
        if "\n\n###" in combined_text:
            instruction, output = combined_text.split("\n\n###")
        else:
            instruction, output = combined_text, ""

        text = f"{instruction.strip()} {EOS_TOKEN} {output.strip()}"
        texts.append(text)
    return {
        "text": texts,
    }

# 데이터셋에 formatting_prompts_func 적용
formatted_dataset = dataset.map(
    formatting_prompts_func,
    batched=True,
)

# 토크나이즈 함수 정의
def tokenize_function(examples):
    texts = examples["text"]
    # 개별 텍스트가 문자열인지 확인
    if isinstance(texts, list) and all(isinstance(t, str) for t in texts):
        tokenized = tokenizer(
            texts,
            padding="max_length",
            truncation=True,
            max_length=512,
            return_tensors="pt",
        )
        return {
            "input_ids": tokenized["input_ids"],
            "attention_mask": tokenized["attention_mask"]
        }
    else:
        raise ValueError("Unexpected input format: Expected a list of strings.")

# 데이터셋에 토큰화 함수 적용
tokenized_dataset = formatted_dataset.map(
    tokenize_function,
    batched=True,
    remove_columns=formatted_dataset.column_names,
)

# 이후 tokenized_dataset을 사용하여 train() 함수를 실행


Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

# **3. Setting Configs**

In [ ]:
# LoRA 설정
peft_params = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.05,
    r=32,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules = ["q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",],
)

# **4 파인튜닝 설정 및 시작 (캐시 비우기 선택)**

In [ ]:
import gc
import torch

# 가비지 컬렉션 수행
gc.collect()
# PyTorch 캐시 비우기
torch.cuda.empty_cache()

In [ ]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

# **4-2. 파인튜닝 시작**

In [ ]:
from trl import SFTConfig, SFTTrainer
from transformers import TrainingArguments

# SFTConfig 설정
sft_config = SFTConfig(
    output_dir="./results",
    dataset_text_field="input_ids",  # 데이터셋의 텍스트 필드 이름을 'input_ids'로 변경
    dataset_num_proc=2,              # 데이터셋 처리에 사용할 프로세스 수
    max_seq_length=512,             # 시퀀스의 최대 길이, 토크나이저와 동일하게 설정합니다.
    num_train_epochs=1,              # 에포크 수 설정
    per_device_train_batch_size=1,
    gradient_accumulation_steps=16,  # 8
    optim="adamw_8bit",              # 최적화 알고리즘
    save_steps=25,
    logging_steps=10,
    learning_rate=2e-5,              # 학습률
    warmup_steps=250,
    weight_decay=0.01,               # 가중치 감소
    fp16=False,
    bf16=False,
    max_grad_norm=1.0,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="cosine",      # 학습률 스케줄러 유형 # 공식은 linear
    report_to="none",
    gradient_checkpointing=True,     # Gradient Checkpointing 활성화
    remove_unused_columns=False,     # 예상치 못한 열 제거를 방지하기 위해 명시적으로 False로 설정합니다.
)

# SFTTrainer 초기화
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=tokenized_dataset,
    peft_config=peft_params,
    args=sft_config,
)

In [ ]:
# 학습 시작
trainer.train()

Step,Training Loss
10,5.471600
20,5.479200
30,5.463100
40,5.394400
50,5.243000
60,5.145500
70,4.994500
80,4.860100
90,4.533000
100,4.309000


/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/

Step,Training Loss
10,5.471600
20,5.479200
30,5.463100
40,5.394400
50,5.243000
60,5.145500
70,4.994500
80,4.860100
90,4.533000
100,4.309000


/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/

TrainOutput(global_step=312, training_loss=3.4653574243569985, metrics={'train_runtime': 8110.4627, 'train_samples_per_second': 0.616, 'train_steps_per_second': 0.038, 'total_flos': 1.1382489555664896e+17, 'train_loss': 3.4653574243569985, 'epoch': 0.9984})

In [ ]:
#학습한 모델 저장
savePath ="./CareerBoost-EXAONE3.0"
trainer.save_model(savePath)

In [ ]:
make_model = "./CareerBoost-EXAONE3.0"

# **5. 허깅페이스 모델 업로드**

In [ ]:
# 머지 모델 - 어뎁터
modelData = PeftModel.from_pretrained(model, make_model)

In [ ]:
# 머지 모델 업로드 준비
mergedModel = modelData.merge_and_unload()

hfRepoId = "LINKER98/CareerBoost-EXAONE3.0" # 만든 레포 이름

# mergedModel.push_to_hub(hfRepoId, use_temp_dir=False)
# tokenizer.push_to_hub(hfRepoId, use_temp_dir=False)

mergedModel.push_to_hub(hfRepoId)
tokenizer.push_to_hub(hfRepoId)

/usr/local/lib/python3.10/dist-packages/peft/tuners/lora/bnb.py:336: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.29G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.18k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/LINKER98/CareerBoost-EXAONE3.0/commit/1523cd6016c7390130f9d6fd2e86495303e64b91', commit_message='Upload tokenizer', commit_description='', oid='1523cd6016c7390130f9d6fd2e86495303e64b91', pr_url=None, repo_url=RepoUrl('https://huggingface.co/LINKER98/CareerBoost-EXAONE3.0', endpoint='https://huggingface.co', repo_type='model', repo_id='LINKER98/CareerBoost-EXAONE3.0'), pr_revision=None, pr_num=None)

# **6. 양자화 및 재배포**

# **7. Loading Pre-trained Model + Test**

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel, LoraConfig

# Google Drive에 저장된 모델 로드
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel, LoraConfig
from google.colab import drive

# 저장된 모델 경로 (Google Drive의 origin 폴더)
model_path = "/content/gdrive/MyDrive/Colab Notebooks/PITer/origin"

# 모델과 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(model_path, trust_remote_code=True)

# 모델 로드 확인
print("Model and tokenizer successfully loaded from:", model_path)

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

Model and tokenizer successfully loaded from: /content/gdrive/MyDrive/Colab Notebooks/PITer/origin


In [ ]:
pwd

'/content/gdrive/MyDrive/Colab Notebooks/PITer'

In [ ]:
from peft import PeftModel

# 모델과 어댑터 로컬 경로 설정
model_path = "./origin"  # 원본 모델 경로
adapter_path = "./PITer-EXAONE"  # adapter_model.safetensors와 adapter_config.json이 있는 경로

# 원본 모델 로드
from transformers import AutoModelForCausalLM, AutoTokenizer
model = AutoModelForCausalLM.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

# 어댑터 로드
model = PeftModel.from_pretrained(model, adapter_path, is_trainable=False)


The repository for ./origin contains custom code which must be executed to correctly load the model. You can inspect the repository content at https://hf.co/./origin.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


ExaoneForCausalLM has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.


Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

In [ ]:
# 프롬프트에 입력에 대한 결과 출력 함수 선언

def text_generation(prompt):

  messages = [
    {"role": "system",
     "content": "You are EXAONE model from LG AI Research, a helpful assistant."},
    {"role": "user", "content": prompt}
  ]

  input_ids = tokenizer.apply_chat_template(
      messages,
      tokenize=True,
      add_generation_prompt=True,
      return_tensors="pt"
      # return_tensors="np"
  )

  output = mergedModel.generate(
      input_ids.to("cuda"),
      eos_token_id=tokenizer.eos_token_id,
      max_new_tokens=512
  )

  print(tokenizer.decode(output[0], skip_special_tokens=True))

In [ ]:
from sklearn.metrics import accuracy_score

# 평가를 위한 샘플 프롬프트와 기대하는 응답 쌍
evaluation_data = [
    {"prompt": "허나 짐의 점심을 조금 주겠소. 새우 부리토요. 이 문장을 정중한 말투로 바꿔줘.", "expected": "하지만 제 점심을 조금 드릴게요. 새우 부리토입니다."},
    {"prompt": "난 지금 뮤지컬 보고 오는 길임 이 문장을 정중한 말투로 바꿔줘.", "expected": "저는 지금 뮤지컬을 보고 돌아오는 길입니다."},
    # 추가적인 테스트 데이터...
]

# 모델의 응답과 기대하는 응답 비교
def evaluate_model():
    correct = 0
    total = len(evaluation_data)

    for data in evaluation_data:
        prompt = data["prompt"]
        expected = data["expected"]

        messages = [
            {"role": "system",
             "content": "You are EXAONE model from LG AI Research, a helpful assistant."},
            {"role": "user", "content": prompt}
        ]

        input_ids = tokenizer.apply_chat_template(
            messages,
            tokenize=True,
            add_generation_prompt=True,
            return_tensors="pt"
        )

        output = model.generate(
            input_ids.to("cpu"),
            eos_token_id=tokenizer.eos_token_id,
            max_new_tokens=50
        )

        generated_text = tokenizer.decode(output[0], skip_special_tokens=True)

        # 간단한 비교를 통해 모델의 정답률 측정
        if expected.lower() in generated_text.lower():
            correct += 1

    accuracy = correct / total
    print(f"Model Accuracy: {accuracy * 100:.2f}%")

# 평가 함수 호출
evaluate_model()

Model Accuracy: 50.00%


In [ ]:
text_generation("ㅎㅇ 요즘 뭐하고 지내냐? 이 문장을 정중한 말투로 바꿔줘.")

[|system|]You are EXAONE model from LG AI Research, a helpful assistant.
[|user|]ᄒᄋ 요즘 뭐하고 지내냐? 이 문장을 정중한 말투로 바꿔줘.
[|assistant|]안녕하세요. 요즘 어떻게 지내고 계신가요?

This sentence in Korean is changed to a more polite form:

안녕하세요. 요즘 어떻게 지내시나요?

I've made the informal 'ᄒᄋ' more formal by using '안녕하세요' instead of '안녕'. I've also changed '뭐하고 지내냐?' to '어떻게 지내시나요?' to make it more polite and respectful.


In [ ]:
text_generation("진짜 괘씸하고 위험한 장난이다.. ㅠ 이 문장을 정중한 말투로 바꿔줘.")

[|system|]You are EXAONE model from LG AI Research, a helpful assistant.
[|user|]진짜 괘씸하고 위험한 장난이다.. ᅲ 이 문장을 정중한 말투로 바꿔줘.
[|assistant|]정말 괘씸하고 위험한 장난이네요. ᅲ


In [ ]:
text_generation("ㅇㅇ 항상 빨간 건 아니고 인터뷰 맞춰서 염색한 거임 이 문장을 정중한 말투로 바꿔줘.")

[|system|]You are EXAONE model from LG AI Research, a helpful assistant.
[|user|]ᄋᄋ 항상 빨간 건 아니고 인터뷰 맞춰서 염색한 거임 이 문장을 정중한 말투로 바꿔줘.
[|assistant|]네, 알겠습니다. 다음과 같이 정중한 말투로 바꿀 수 있습니다:

"항상 빨간 색은 아니며, 인터뷰 일정에 맞춰 염색한 것입니다."

이 문장은 더 공식적이고 정중한 느낌을 줍니다. 도움이 되었길 바랍니다!


In [ ]:
text_generation("허나 짐의 점심을 조금 주겠소. 새우 부리토요. 이 문장을 정중한 말투로 바꿔줘.")

[|system|]You are EXAONE model from LG AI Research, a helpful assistant.
[|user|]허나 짐의 점심을 조금 주겠소. 새우 부리토요. 이 문장을 정중한 말투로 바꿔줘.
[|assistant|]물론입니다. 다음과 같이 정중한 말투로 바꿀 수 있습니다:

"그러나 제가 조금이라도 점심을 드릴 수 있다면, 새우 부리토를 준비해 드리겠습니다."

이 문장은 더 공손하고 정중한 표현으로 변경되었습니다. 도움이 되었기를 바랍니다!


In [ ]:
text_generation("난 지금 뮤지컬 보고 오는 길임 이 문장을 정중한 말투로 바꿔줘.")

[|system|]You are EXAONE model from LG AI Research, a helpful assistant.
[|user|]난 지금 뮤지컬 보고 오는 길임 이 문장을 정중한 말투로 바꿔줘.
[|assistant|]저는 지금 뮤지컬을 보고 돌아오는 길입니다.


In [ ]:
text_generation("무엇보다, 충분히 먹었으면 과거로 되돌아가지 않으면 될 일이지. 무한 개의 치킨 샌드위치를 원하지만 영원히 먹고싶은 마음은 없어~╮(╯▽╰)╭ 이 문장을 정중한 말투로 바꿔줘.")

[|system|]You are EXAONE model from LG AI Research, a helpful assistant.
[|user|]무엇보다, 충분히 먹었으면 과거로 되돌아가지 않으면 될 일이지. 무한 개의 치킨 샌드위치를 원하지만 영원히 먹고싶은 마음은 없어~╮(╯▽╰)╭ 이 문장을 정중한 말투로 바꿔줘.
[|assistant|]물론입니다. 다음과 같이 정중한 말투로 바꿀 수 있습니다:

"무엇보다도, 충분히 먹었다면 과거로 돌아갈 필요는 없겠지요. 무한 개의 치킨 샌드위치를 원하지만, 영원히 먹고 싶은 마음은 없습니다."

이 문장은 더 공손하고 정중한 표현으로 변경되었습니다. 도움이 되었기를 바랍니다!


# **(안쓰는 것)3. Data Preprecessing**

In [ ]:
dataPath = "/content/data/smilestyle_dataset.tsv"

# Open the file in binary mode
with open(dataPath, 'rb') as f:
    # Detect the encoding
    result = chardet.detect(f.read())

dataset = pd.read_csv(dataPath, encoding=result['encoding'], sep="\t", on_bad_lines="warn")

In [ ]:
save_dataset_to_csv(new_dataset, "/content/data/modified_traindata.csv")

In [ ]:
EOS_TOKEN = tokenizer.eos_token

In [ ]:
print(dataset['translator'][3704])

nan


In [ ]:
def generate_prompt(example):
  output_texts = []
  #appending
  for i in range(len(example['chat'])):
    prompt = f"### Instruction: {example['chat'][i]} 이 문장을 정중한 말투로 바꿔줘.\n\n### Response: {example['formal'][i]}"
    prompt = prompt + EOS_TOKEN
    output_texts.append(prompt)

  for i in range(len(example['choding'])):
    prompt = f"### Instruction: {example['choding'][i]} 이 문장을 정중한 말투로 바꿔줘.\n\n### Response: {example['formal'][i]}"
    prompt = prompt + EOS_TOKEN
    output_texts.append(prompt)

  for i in range(len(example['sosim'])):
    prompt = f"### Instruction: {example['sosim'][i]} 이 문장을 정중한 말투로 바꿔줘.\n\n### Response: {example['formal'][i]}"
    prompt = prompt + EOS_TOKEN
    output_texts.append(prompt)

  for i in range(len(example['translator'])):
    prompt = f"### Instruction: {example['translator'][i]} 이 문장을 정중한 말투로 바꿔줘.\n\n### Response: {example['formal'][i]}"
    prompt = prompt + EOS_TOKEN
    output_texts.append(prompt)

  return output_texts

In [ ]:
new_dataset = generate_prompt(dataset)

In [ ]:
new_dataset

['### Instruction: 하잉ㅋㅋ 나 떼걸룩 6마리 키운다! 이 문장을 정중한 말투로 바꿔줘.\n\n### Response: 안녕하세요. 저는 고양이 6마리 키워요.[|endofturn|]',
 '### Instruction: 엥? 6마리나? 안힘듬?ㅋㅋㅋㅋ 이 문장을 정중한 말투로 바꿔줘.\n\n### Response: 고양이를 6마리나요? 키우는거 안 힘드세요?[|endofturn|]',
 '### Instruction: 내가 고양이 좋아해서 딱히 안힘듬 ㅋㅋㅋ 이 문장을 정중한 말투로 바꿔줘.\n\n### Response: 제가 워낙 고양이를 좋아해서 크게 힘들진 않아요.[|endofturn|]',
 '### Instruction: 가장 나이 먹은 고양이가 몇살이야? 이 문장을 정중한 말투로 바꿔줘.\n\n### Response: 가장 나이가 많은 고양이가 어떻게 돼요?[|endofturn|]',
 '### Instruction: 이제 여섯살이고 갈색고양이임! 이 문장을 정중한 말투로 바꿔줘.\n\n### Response: 여섯 살입니다. 갈색 고양이에요.[|endofturn|]',
 '### Instruction: 그럼 가장 애기인 고양이는 몇살이야? 이 문장을 정중한 말투로 바꿔줘.\n\n### Response: 그럼 가장 어린 고양이가 어떻게 돼요?[|endofturn|]',
 '### Instruction: 작년에 분양 받았는데 1살이야! 이 문장을 정중한 말투로 바꿔줘.\n\n### Response: 한 살입니다. 작년에 분양 받았어요.[|endofturn|]',
 '### Instruction: 냥이들끼리 안 싸움? 이 문장을 정중한 말투로 바꿔줘.\n\n### Response: 그럼 고양이들끼리 안 싸우나요?[|endofturn|]',
 '### Instruction: 우리 일곱은 다 한 가족이라서 싸울 일이 없어 이 문장을 정중한 말투로 바꿔줘.\n\n### Response: 저희 일곱은 다같이 한 가족입니다. 싸우는 일은 없어요.

In [ ]:
print(new_dataset[4000])

### Instruction: ㅎㅇ 요즘 뭐하고 지내냐? 이 문장을 정중한 말투로 바꿔줘.

### Response: 안녕하세요. 요즘은 어떻게 지내시나요?[|endofturn|]


In [ ]:
print(new_dataset[7706])

### Instruction: 별로 상황이 좋지는 않아.. 지난달에 은행에서 잘렸거든 ㅠ 이 문장을 정중한 말투로 바꿔줘.

### Response: 별로 상황이 좋지는 않습니다. 지난 달에 은행에서 잘렸어요.[|endofturn|]


In [ ]:
def save_dataset_to_csv(dataset, file_path):
    """
    주어진 데이터셋을 CSV 파일로 저장합니다.

    매개변수:
    - dataset: 리스트의 리스트 형태로, 각 내부 리스트는 데이터의 한 행을 나타냅니다.
    - file_path: 저장할 CSV 파일의 경로와 이름을 포함한 문자열입니다.

    반환값:
    - 없음
    """
    # 데이터셋을 DataFrame으로 변환
    df = pd.DataFrame(dataset)

    # DataFrame을 CSV 파일로 저장
    df.to_csv(file_path, index=False, encoding='utf-8-sig')

In [ ]:
save_dataset_to_csv(new_dataset, "/content/data/modified_traindata.csv")

In [ ]:
train_data = load_dataset("csv", data_files="/content/data/traindata.csv")

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
pwd

'/content'

In [ ]:
train_data

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 10219
    })
})

In [ ]:
print(train_data['train'][4001]['text'])

### Instruction: 어떤 거 배움?? 이 문장을 정중한 말투로 바꿔줘.

### Response: 어떤 사실들을 배웠나요?[|endofturn|]
